In [2]:
import os

# Get current working directory
current_path = os.getcwd()

# Print the path
print(current_path)

def submit_slurm_job(slurm_script_content, script_filename="submit_job.sh"):
    """
    Writes a Slurm script to a file and submits it using sbatch.

    Args:
        slurm_script_content (str): A string containing the entire Slurm script.
        script_filename (str, optional): The name of the file to write the script to. 
                                         Defaults to "submit_job.sh".

    Returns:
        bool: True if the job was submitted successfully, False otherwise.
    """
    # Write the string content to the specified .sh file
    try:
        with open(script_filename, "w") as f:
            f.write(slurm_script_content)
    except IOError as e:
        print(f"❌ Error writing script file: {e}")
        return False

    # Use subprocess to run the sbatch command
    try:
        # The 'check=True' argument will raise an error if sbatch fails
        result = subprocess.run(
            ["sbatch", script_filename],
            capture_output=True,
            text=True,
            check=True
        )
        print("✅ Job submitted successfully!")
        print(result.stdout)  # This usually contains the job ID
        return True
    except FileNotFoundError:
        print("❌ Error: 'sbatch' command not found.")
        print("Please ensure you are on a system with Slurm installed.")
        return False
    except subprocess.CalledProcessError as e:
        print("❌ Error submitting job via sbatch:")
        print(e.stderr)  # Print the error message from Slurm
        return False
    finally:
        # The script file is kept for inspection but you could remove it.
        # To clean up, uncomment the following line:
        #os.remove(script_filename)
        pass

/gpfs/home4/bryanc


In [9]:
import subprocess
import os

# Define the Slurm script as a multi-line string
slurm_script_content = """#!/bin/bash
#SBATCH --job-name=rfdiffusion_scaffold    # Name of the job
#SBATCH --nodes=1                          # Request a single node
#SBATCH --time=00:10:00                    # Maximum execution time: 20 minutes
#SBATCH --partition=gpu_a100               # Request the A100 GPU partition
#SBATCH --gpus=1                           # Request one GPU
#SBATCH --ntasks=1                         # Run a single task
#SBATCH --cpus-per-task=8                  # Allocate 8 CPUs for the task
#SBATCH --output=rfdiffusion_%j.out        # Standard output log
#SBATCH --error=rfdiffusion_%j.err         # Standard error log

# Define the base project directory
PROJECT_SPACE="/projects/0/prjs0823/apptainers"

echo "$PROJECT_SPACE"

# Paths for the RFdiffusion container and input files
RFD_CONTAINER_PATH="$PROJECT_SPACE/rfdiffusion/containers/rfdiffusion-cuda11.8.sif"
INPUT_PDB_PATH="$PROJECT_SPACE/rfdiffusion/examples/5TPN.pdb"
OUTPUT_PATH="$PROJECT_SPACE/out/"
OVERLAY_FILE="$PROJECT_SPACE/rfdiffusion_overlay.img"

# Create the output directory if it doesn't exist
mkdir -p "$OUTPUT_PATH"

# Create an overlay file for writable storage inside the container
# We add --force to overwrite an existing overlay file from a previous run
apptainer overlay create --size 128 "$OVERLAY_FILE"

# Run RFdiffusion using an Apptainer container
apptainer run --nv \\
    --overlay "$OVERLAY_FILE" \\
    -B "$PWD:/workspace" \\
    --pwd /workspace \\
    "$RFD_CONTAINER_PATH" \\
    inference.output_prefix="$OUTPUT_PATH/design" \\
    inference.input_pdb="$INPUT_PDB_PATH" \\
    inference.num_designs=1 \\
    'contigmap.contigs=[10-40/A163-181/10-40]' \\
    inference.model_directory_path=/app/RFdiffusion/models
"""

# Define the name for the temporary script file
submit_slurm_job(slurm_script_content, script_filename="submit_job_rfd.job")

✅ Job submitted successfully!
Submitted batch job 14062594



True